In [ ]:
from datapile import HuggingfaceLoader
data = HuggingfaceLoader(
    dataset_path="/home/user/ckwan1/ml/huggingface_dataset", 
    batch_size=1, 
    shuffle=False,
    num_workers=1,
)
data.setup("validation")
val_iter = iter(data.val_dataloader())
za_tensor, fastpm_tensor = next(val_iter)


/home/user/ckwan1/.conda/envs/d3m/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from model_backbone import UNet3D, BasicBlock
from torchinfo import summary

UNet3D = UNet3D(BasicBlock, num_layers=3, base_filters=64, blocks_per_layer=2, 
                 init_dim=3)
x_shape = (1 ,3 ,32 ,32 ,32)
summary(UNet3D, x_shape)

Layer (type:depth-idx)                   Output Shape              Param #
UNet3D                                   [1, 3, 32, 32, 32]        --
├─Sequential: 1-1                        [1, 64, 32, 32, 32]       --
│    └─BasicBlock: 2-1                   [1, 64, 32, 32, 32]       --
│    │    └─Conv3d: 3-1                  [1, 64, 32, 32, 32]       5,248
│    │    └─ReLU: 3-2                    [1, 64, 32, 32, 32]       --
│    └─BasicBlock: 2-2                   [1, 64, 32, 32, 32]       --
│    │    └─Conv3d: 3-3                  [1, 64, 32, 32, 32]       110,656
│    │    └─ReLU: 3-4                    [1, 64, 32, 32, 32]       --
├─ModuleList: 1-2                        --                        --
│    └─Sequential: 2-3                   [1, 128, 16, 16, 16]      --
│    │    └─BasicBlock: 3-5              [1, 128, 16, 16, 16]      221,312
│    └─Sequential: 2-4                   [1, 128, 16, 16, 16]      --
│    │    └─BasicBlock: 3-6              [1, 128, 16, 16, 16]      442,4

In [9]:
from magvit import MagVitAE3D
from torchinfo import summary

magvit = MagVitAE3D(n_bands = 3,
        hidden_dims = 512,
        residual_conv_kernel_size = 3,
        n_compressions = 3,
        num_consecutive = 3,)
x_shape = (1 ,3 ,32 ,32 ,32)
summary(magvit, x_shape)

/home/user/ckwan1/.conda/envs/d3m/lib/python3.12/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Layer (type:depth-idx)                   Output Shape              Param #
MagVitAE3D                               [1, 3, 32, 32, 32]        --
├─Conv3d: 1-1                            [1, 64, 32, 32, 32]       65,920
├─ModuleList: 1-2                        --                        --
│    └─Residual3D: 2-1                   [1, 64, 32, 32, 32]       --
│    │    └─Sequential: 3-1              [1, 64, 32, 32, 32]       114,816
│    └─SpatialDownsample3D: 2-2          [1, 128, 16, 16, 16]      --
│    │    └─Conv3d: 3-2                  [1, 128, 16, 16, 16]      221,312
│    └─Sequential: 2-3                   [1, 128, 16, 16, 16]      --
│    │    └─Residual3D: 3-3              [1, 128, 16, 16, 16]      459,008
│    │    └─Residual3D: 3-4              [1, 128, 16, 16, 16]      459,008
│    │    └─Residual3D: 3-5              [1, 128, 16, 16, 16]      459,008
│    └─SpatialDownsample3D: 2-4          [1, 256, 8, 8, 8]         --
│    │    └─Conv3d: 3-6                  [1, 256, 8, 8, 

In [12]:
from model import VAE3DwithRes, BasicBlock
from torchinfo import summary

VAE3DwithRes = VAE3DwithRes(BasicBlock, num_layers=4, base_filters=64, blocks_per_layer=2, 
                 init_dim=3, latent_dim=1024)
x_shape = (1 ,3 ,32 ,32 ,32)
summary(VAE3DwithRes, x_shape)

Layer (type:depth-idx)                   Output Shape              Param #
VAE3DwithRes                             [1, 3, 32, 32, 32]        --
├─Sequential: 1-1                        [1, 64, 32, 32, 32]       --
│    └─BasicBlock: 2-1                   [1, 64, 32, 32, 32]       --
│    │    └─Conv3d: 3-1                  [1, 64, 32, 32, 32]       5,248
│    │    └─BatchNorm3d: 3-2             [1, 64, 32, 32, 32]       128
│    │    └─ReLU: 3-3                    [1, 64, 32, 32, 32]       --
│    └─ResBlock: 2-2                     [1, 64, 32, 32, 32]       --
│    │    └─Conv3d: 3-4                  [1, 64, 32, 32, 32]       110,656
│    │    └─BatchNorm3d: 3-5             [1, 64, 32, 32, 32]       128
│    │    └─ReLU: 3-6                    [1, 64, 32, 32, 32]       --
├─ModuleList: 1-2                        --                        --
│    └─Sequential: 2-3                   [1, 128, 16, 16, 16]      --
│    │    └─BasicBlock: 3-7              [1, 128, 16, 16, 16]      221,568


In [1]:
from srvae import srVAE3D
from srvae_prior import *
import torch
from torchinfo import summary

x_shape = (1 ,3 ,32 ,32 ,32)
y_shape = (1, 3 ,32 ,32 ,32)
model = srVAE3D((3 ,32 ,32 ,32),(3 ,32 ,32 ,32),1024,1024,StandardNormal)
x_tensor = torch.randn(x_shape)
y_tensor = torch.randn(y_shape)
summary(model, input_data=[x_tensor, y_tensor])


Layer (type:depth-idx)                                                                     Output Shape              Param #
srVAE3D                                                                                    [1, 3, 32, 32, 32]        --
├─q_u: 1-1                                                                                 [1, 2, 16, 16, 16]        --
│    └─Sequential: 2-1                                                                     [1, 4, 16, 16, 16]        --
│    │    └─DenselyEncoder3D: 3-1                                                          [1, 4, 16, 16, 16]        9,705,764
├─q_z: 1-2                                                                                 [1, 2, 8, 8, 8]           --
│    └─Sequential: 2-2                                                                     [1, 4, 8, 8, 8]           --
│    │    └─DenselyEncoder3D: 3-2                                                          [1, 4, 8, 8, 8]           7,187,166
├─p_x: 1-3           

In [6]:
from score_model import UNet3DModel
from torchinfo import summary
import torch
UNet3DModel = UNet3DModel(image_size=32,act_f='swish')
x_shape = (1 ,6 ,32 ,32 ,32)
t = (1)
x_tensor = torch.randn(x_shape)
t_tensor = torch.randn(t)
summary(UNet3DModel, input_data=[x_tensor, t_tensor])

Layer (type:depth-idx)                        Output Shape              Param #
UNet3DModel                                   [1, 3, 32, 32, 32]        --
├─ModuleList: 1-1                             --                        --
│    └─GaussianFourierProjection: 2-1         [1, 64]                   (32)
│    └─Linear: 2-2                            [1, 128]                  8,320
│    └─ResnetBlockBigGANpp: 2-239             --                        (recursive)
│    │    └─SiLU: 3-1                         [1, 128]                  --
│    └─Linear: 2-4                            [1, 128]                  16,512
│    └─Conv3d: 2-5                            [1, 32, 32, 32, 32]       5,216
│    └─ResnetBlockBigGANpp: 2-6               [1, 32, 32, 32, 32]       59,552
│    │    └─GroupNorm: 3-2                    [1, 32, 32, 32, 32]       64
│    └─ResnetBlockBigGANpp: 2-239             --                        (recursive)
│    │    └─SiLU: 3-3                         [1, 32, 32, 32,

In [10]:
from naf_net import NAFNet3D_base
from torchinfo import summary
import torch
NAFNet3D_base = NAFNet3D_base(width=64,enc_blk_nums=[2,2,4,8],middle_blk_num=6,dec_blk_nums=[2,2,2,2],dw_expand=2,ffn_expand=2)
x_shape = (1 ,3 ,32 ,32 ,32)
x_tensor = torch.randn(x_shape)
summary(NAFNet3D_base, input_data=x_tensor)

Layer (type:depth-idx)                             Output Shape              Param #
NAFNet3D_base                                      [1, 3, 32, 32, 32]        --
├─Conv3d: 1-1                                      [1, 64, 32, 32, 32]       5,248
├─ModuleList: 1-8                                  --                        (recursive)
│    └─Sequential: 2-1                             [1, 64, 32, 32, 32]       --
│    │    └─BaselineBlock: 3-1                     [1, 64, 32, 32, 32]       53,696
│    │    └─BaselineBlock: 3-2                     [1, 64, 32, 32, 32]       53,696
├─ModuleList: 1-9                                  --                        (recursive)
│    └─Conv3d: 2-2                                 [1, 128, 16, 16, 16]      65,664
├─ModuleList: 1-8                                  --                        (recursive)
│    └─Sequential: 2-3                             [1, 128, 16, 16, 16]      --
│    │    └─BaselineBlock: 3-3                     [1, 128, 16, 16, 16]  

In [6]:
from naf_net import NAFNet3D
from torchinfo import summary
import torch
NAFNet3D = NAFNet3D(width=64,enc_blk_nums=[2,2,4,8],middle_blk_num=12,dec_blk_nums=[2,2,2,2])
x_shape = (1 ,3 ,32 ,32 ,32)
x_tensor = torch.randn(x_shape)
summary(NAFNet3D, input_data=x_tensor)

Layer (type:depth-idx)                             Output Shape              Param #
NAFNet3D                                           [1, 3, 32, 32, 32]        --
├─Conv3d: 1-1                                      [1, 64, 32, 32, 32]       5,248
├─ModuleList: 1-8                                  --                        (recursive)
│    └─Sequential: 2-1                             [1, 64, 32, 32, 32]       --
│    │    └─BaselineBlock: 3-1                     [1, 64, 32, 32, 32]       31,264
│    │    └─BaselineBlock: 3-2                     [1, 64, 32, 32, 32]       31,264
├─ModuleList: 1-9                                  --                        (recursive)
│    └─Conv3d: 2-2                                 [1, 128, 16, 16, 16]      65,664
├─ModuleList: 1-8                                  --                        (recursive)
│    └─Sequential: 2-3                             [1, 128, 16, 16, 16]      --
│    │    └─BaselineBlock: 3-3                     [1, 128, 16, 16, 16]  